In [ ]:
import json
import math
import time
import pandas as pd
from simul import node, edge, port, AMHS

# 레이아웃 데이터 로드
with open("fab_oht_layout_updated.json") as f:
    layout_data = json.load(f)

nodes = [node(n['id'], [n['x'], n['y']]) for n in layout_data['nodes']]

edges = [
    edge(
        source=next(node for node in nodes if node.id == rail['from']),
        dest=next(node for node in nodes if node.id == rail['to']),
        length=math.sqrt(
            (next(node for node in nodes if node.id == rail['to']).coord[0] -
             next(node for node in nodes if node.id == rail['from']).coord[0])**2 +
            (next(node for node in nodes if node.id == rail['to']).coord[1] -
             next(node for node in nodes if node.id == rail['from']).coord[1])**2
        ),
        max_speed=1000 if rail.get('curve', 0) == 1 else 5000
    )
    for rail in layout_data['rails']
]

ports = [
    port(
        name=p['name'],
        from_node=p['from_node'],
        to_node=p['to_node'],
        from_dist=p['distance']
    )
    for p in layout_data['ports']
]

# 🔹 소켓 없이 시뮬만 도는 함수
def run_benchmark(num_ohts=200, max_time=2000, time_step=0.1):
    amhs = AMHS(nodes=nodes, edges=edges, ports=ports, num_OHTs=num_ohts, max_jobs=1000)


    start = time.time()
    current_time = 0
    count = 0
    while current_time <= max_time:
        if count % 100 == 0:
            amhs.generate_job()
        amhs.assign_jobs()
        
        for oht in amhs.OHTs:
            oht.move(time_step, current_time)
        
        amhs.update_edge_metrics(current_time, time_window=60)
        
        for oht in amhs.OHTs:
            oht.cal_pos(time_step)
        
        current_time += time_step
        count += 1
    end = time.time()


    return end - start

# 여러 설정값 테스트
num_ohts_list = [100, 200, 500]
max_time_list = [1000, 2000, 4000]
time_step = 0.1

results = []
for num_ohts in num_ohts_list:
    for max_time in max_time_list:
        elapsed = run_benchmark(num_ohts=num_ohts, max_time=max_time, time_step=time_step)
        results.append({
            "num_ohts": num_ohts,
            "max_time": max_time,
            "time_step": time_step,
            "elapsed_sec": round(elapsed, 2)
        })
        print(f"num_ohts={num_ohts}, max_time={max_time} → {elapsed:.2f} sec")

# 결과를 데이터프레임으로 정리
df = pd.DataFrame(results)
df


num_ohts=100, max_time=1000 → 33.97 sec
num_ohts=100, max_time=2000 → 64.75 sec


KeyboardInterrupt: 